## Analysis of time-to-resolve Get It Done San Diego requests
### Notebook 3: Inferential Statistics Exploration
Most of the statistical tests in this project were performed in Excel. This notebook contains data-wrangling code to extract the appropriate data for individual tests, which were then exported to .csv files for analysis.

Results of these tests are reported in the Word document "Analysis of factors influencing GID closure time.docx"

#### Import necessary libraries

In [1]:
import pandas as pd
from datetime import timedelta
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import os
notebook_path = os.path.abspath("3b-Inferential_Statistics-Exploration.ipynb")

In [2]:
filename_csv = os.path.join(os.path.dirname(notebook_path), "data/gid_with_demo.csv")
gid_reduced = pd.read_csv(filename_csv, index_col=0)

In [3]:
# Add three more columns in order to categorize 'days_till_closed' in addtion
# to having it as a continuous variable
gid_reduced['closed_in_14days'] = 0
gid_reduced.loc[gid_reduced['days_till_closed'] < 15, 'closed_in_14days'] = 1

gid_reduced['closed_in_45days'] = 0
gid_reduced.loc[(gid_reduced['days_till_closed'] >= 15) & (gid_reduced['days_till_closed'] < 46),
                                   'closed_in_45days'] = 1

gid_reduced['closed_after_45days'] = 0
gid_reduced.loc[(gid_reduced['days_till_closed'] >= 45), 'closed_after_45days'] = 1

gid_reduced['closed_after_60days'] = 0
gid_reduced.loc[(gid_reduced['days_till_closed'] >= 60), 'closed_after_60days'] = 1

print(len(gid_reduced))
gid_reduced.head()

305524


,service_request_id,requested_datetime,service_name,case_record_type,updated_datetime,status,lat,long,district,case_origin,...,load_by_service,load_by_service_zip,household_income,housing_value,median_age,pop_density,closed_in_14days,closed_in_45days,closed_after_45days,closed_after_60days
0,80478,2016-05-20 05:04:00,PARKING METER,TSW,NaN,Closed - Referred,32.726553,-117.125296,3,Mobile,...,0,0,40605.0,375299.0,30.2,9334.843581,0,0,0,0
1,80479,2016-05-20 05:05:00,72 HOUR VIOLATION,Parking,NaN,Closed - Referred,32.726523,-117.125532,3,Mobile,...,0,0,40605.0,375299.0,30.2,9334.843581,0,0,0,0
2,80480,2016-05-20 05:05:00,DEAD ANIMAL,TSW,NaN,Closed - Referred,32.726365,-117.125478,3,Mobile,...,0,0,40605.0,375299.0,30.2,9334.843581,0,0,0,0
3,80481,2016-05-20 05:06:00,GRAFFITI REMOVAL,TSW,2016-05-20 07:39:00,Closed,32.726445,-117.125476,3,Mobile,...,0,0,40605.0,375299.0,30.2,9334.843581,1,0,0,0
4,80482,2016-05-20 05:06:00,ILLEGAL DUMPING,ESD Complaint/Report,NaN,Closed - Referred,32.727625,-117.124705,3,Mobile,...,0,0,40605.0,375299.0,30.2,9334.843581,0,0,0,0


### Testing correlations between percent mobile/web/phone access and demograpics. 
Here we use all submitted requests (regardless of status), and group by zipcode. We finish up with a multiple linear regression model to determine which factors are most important. A lot of statistics will be done in Excel, so sections will end with a saved CSV file.
1. Extract number of requests made via mobile app, phone, and website, and calculate percentage of each
2. Add demographic information
3. Save to stats_table_1.csv

In [4]:
# Extract number of requests from only mobile, phone, 
# and web requests grouped by district
temp_summary = pd.DataFrame()
temp_summary = gid_reduced[(gid_reduced.case_origin=='Mobile') | (gid_reduced.case_origin=='Phone') |
                         (gid_reduced.case_origin=='Web')].groupby('zipcode').case_origin.value_counts()
temp_summary = temp_summary.unstack(1)

# Calculate ratio of mobile, phone, and web requests 
# from the sum of those requests only
temp_summary.loc[:,"Mobile":"Web"] = temp_summary.loc[:,"Mobile":"Web"].div(temp_summary.sum(axis=1), axis=0)

# Second, add columns for household income and age, averaged over district
temp_summary['household_income'] = gid_reduced.groupby('zipcode').household_income.mean()
temp_summary['housing_value'] = gid_reduced.groupby('zipcode').housing_value.mean()
temp_summary['median_age'] = gid_reduced.groupby('zipcode').median_age.mean()
temp_summary['pop_density'] = gid_reduced.groupby('zipcode').pop_density.mean()
temp_summary.head()

#temp_summary.to_csv('stats_table_1.csv', header=True)

# In case you want to look at linear regression within this notebook:
#m = ols('Mobile ~ household_income + pop_density + median_age + housing_value',temp_summary).fit()
#print(m.summary())
#print("RSS:",m.ssr)


case_origin,Mobile,Phone,Web,household_income,housing_value,median_age,pop_density
zipcode,,,,,,,
91902,0.554810,0.118568,0.326622,76075.0,595112.0,41.3,2024.891030
91911,0.381791,0.151742,0.466467,50186.0,374794.0,32.9,7086.058226
91932,0.406968,0.111164,0.481868,45790.0,410020.0,30.6,10233.983290
91942,0.342003,0.158648,0.499350,49556.0,412411.0,37.5,6525.368529
91945,0.568807,0.151682,0.279511,51932.0,383639.0,34.9,6488.277268


### Testing correlations between timely-closure rate and demographics
Here we only look at closed requests, and calculate the percentage that are closed within 45 days. Group by zipcode, and correlate with demographic data (multiple regression model).
1. Extract closed requests only
2. Remove employee-initiated "Graffiti Removal" requests (because those are all closed immediately--I think they are entered into the database *after* they've been done
3. Calculate percentage that have days_till-closed < 45
4. Add demographic information
5. Save to stats_table_2.csv

In [5]:
# Extract closed requests
tmp_closed = gid_reduced[gid_reduced['status']=='Closed']

# Remove Crew/Self Generated Graffiti Removal requests
tmp_closed = tmp_closed.drop(tmp_closed[(tmp_closed.case_origin == 'Crew/Self Generated') & 
                             (tmp_closed.service_name == 'Graffiti Removal')].index)

# Calculate timely-closed by zipcode
temp_summary = pd.DataFrame()
temp_summary['timely_closure'] = tmp_closed.groupby('zipcode').closed_in_45days.sum() / tmp_closed.groupby('zipcode').closed_in_45days.size()

# Add columns for household income and age, averaged over district
temp_summary['household_income'] = gid_reduced.groupby('zipcode').household_income.mean()
temp_summary['housing_value'] = gid_reduced.groupby('zipcode').housing_value.mean()
temp_summary['median_age'] = gid_reduced.groupby('zipcode').median_age.mean()
temp_summary['pop_density'] = gid_reduced.groupby('zipcode').pop_density.mean()
temp_summary.head()
#temp_summary.to_csv('stats_table_2.csv', header=True)

# If you want to look at linear regression within the notebook
#m = ols('timely_closure ~ pop_density',temp_summary).fit()
#print(m.summary())
#print("RSS:",m.ssr)

,timely_closure,household_income,housing_value,median_age,pop_density
zipcode,,,,,
91902,0.318612,76075.0,595112.0,41.3,2024.891030
91911,0.238629,50186.0,374794.0,32.9,7086.058226
91932,0.244411,45790.0,410020.0,30.6,10233.983290
91942,0.212617,49556.0,412411.0,37.5,6525.368529
91945,0.196234,51932.0,383639.0,34.9,6488.277268


### Testing differences between timely-closure rate and request method 
Here we only look at closed requests, and look at how many requests are closed within 45 days according to request method (for test of homogeneity of proportions)
1. Extract closed requests only
2. Remove employee-initiated "Graffiti Removal" requests
3. Count number of timely closures and total requests across request method
4. Save to stats_table_3.csv

In [6]:
# Extract closed requests
tmp_closed = gid_reduced[gid_reduced['status']=='Closed']

# Remove Crew/Self Generated Graffiti Removal requests
tmp_closed = tmp_closed.drop(tmp_closed[(tmp_closed.case_origin == 'Crew/Self Generated') & 
                             (tmp_closed.service_name == 'Graffiti Removal')].index)

# Calculate timely-closed by case_origin
temp_summary = pd.DataFrame()
temp_summary['timely_closure'] = (tmp_closed.groupby('case_origin').closed_in_14days.sum() + 
                                  tmp_closed.groupby('case_origin').closed_in_45days.sum())
temp_summary['total_requests'] = tmp_closed.groupby('case_origin').closed_in_45days.size()
temp_summary
#temp_summary.to_csv('stats_table_3.csv', header=True)

,timely_closure,total_requests
case_origin,,
999,22,22
Cost Schedule,1,9
Crew/Self Generated,11061,12491
Deficiency Report,4,11
Email,1449,3540
Garage Run,3,4
Internal,29,31
Letter,3,7
Mobile,62762,77495


### ANOVA to compare days_till_closed across request methods

1. Extract closed requests only
2. Remove employee-initiated "Graffiti Removal" requests
3. Keep only requests closed within 45 days

In [7]:
# Extract closed requests
tmp_closed = gid_reduced[gid_reduced['status']=='Closed']

# Remove Crew/Self Generated Graffiti Removal requests
tmp_closed = tmp_closed.drop(tmp_closed[(tmp_closed.case_origin == 'Crew/Self Generated') & 
                             (tmp_closed.service_name == 'Graffiti Removal')].index)

# Only keep timely-closure cases
tmp_closed = tmp_closed[tmp_closed['closed_after_45days'] == 0]

# Calculate days_till_closed by case_origin
print(tmp_closed[(tmp_closed['case_origin'] == "Email") |
                 (tmp_closed['case_origin'] == "Mobile") |
                 (tmp_closed['case_origin'] == 'Phone') |
                 (tmp_closed['case_origin'] == 'Web')].groupby('case_origin').days_till_closed.describe())

email = np.asarray(tmp_closed[tmp_closed.case_origin=='Email'].days_till_closed)
mobile = np.asarray(tmp_closed[tmp_closed.case_origin=='Mobile'].days_till_closed)
phone = np.asarray(tmp_closed[tmp_closed.case_origin=='Phone'].days_till_closed)
web = np.asarray(tmp_closed[tmp_closed.case_origin=='Web'].days_till_closed)

stats.f_oneway(email, mobile, phone, web)

               count       mean        std       min       25%       50%  \
case_origin                                                                
Email         1439.0  13.912593  12.010737  0.000694  3.938542  9.623611   
Mobile       62519.0  11.378716  10.535571  0.000000  3.553472  7.889583   
Phone        23781.0   9.758624  10.155524  0.000000  2.005556  6.351389   
Web          44249.0  12.442736  10.601839  0.000000  4.315278  9.540278   

                   75%        max  
case_origin                        
Email        22.410764  44.822917  
Mobile       16.303472  44.998611  
Phone        13.909028  44.986806  
Web          17.872917  44.984028  


F_onewayResult(statistic=364.1626869012801, pvalue=1.4636437371982495e-235)

### Look at days_till_closed correlations with demographics
Here we only look at closed requests, and calculate the percentage that are closed within 45 days. Group by zipcode, and correlate with demographic data.
1. Extract closed requests only
2. Remove employee-initiated "Graffiti Removal" requests
3. Calculate mean of days_till_closed by zipcode
4. Save to stats_table_4.csv

In [8]:
# Extract closed requests
tmp_closed = gid_reduced[(gid_reduced['status']=='Closed') &
                         (gid_reduced['closed_after_45days']==0)]

# Remove Crew/Self Generated Graffiti Removal requests
tmp_closed = tmp_closed.drop(tmp_closed[(tmp_closed.case_origin == 'Crew/Self Generated') & 
                             (tmp_closed.service_name == 'Graffiti Removal')].index)

# Calculate days_till_closed by zipcode
temp_summary = pd.DataFrame()
temp_summary['days_till_closed'] = tmp_closed.groupby('zipcode').days_till_closed.mean()

# Add columns for household income and age, averaged over district
temp_summary['household_income'] = gid_reduced.groupby('zipcode').household_income.mean()
temp_summary['housing_value'] = gid_reduced.groupby('zipcode').housing_value.mean()
temp_summary['median_age'] = gid_reduced.groupby('zipcode').median_age.mean()
temp_summary['pop_density'] = gid_reduced.groupby('zipcode').pop_density.mean()
temp_summary.head()

#temp_summary.to_csv('stats_table_4.csv', header=True)

,days_till_closed,household_income,housing_value,median_age,pop_density
zipcode,,,,,
91902,13.265550,76075.0,595112.0,41.3,2024.891030
91911,11.530660,50186.0,374794.0,32.9,7086.058226
91932,11.313965,45790.0,410020.0,30.6,10233.983290
91942,11.423020,49556.0,412411.0,37.5,6525.368529
91945,10.006494,51932.0,383639.0,34.9,6488.277268


#### Look at timely_closure across service_name
(Chi-squared test)
Here we only look at closed requests, and count the number of timely-closures in each service name type (for test of homogeneity of proportions).
1. Extract closed requests only
2. Remove employee-initiated "Graffiti Removal" requests
3. Count number of timely-closed requests, grouped by service_name
4. Save to stats_table_5.csv

In [9]:
# Extract closed requests
tmp_closed = gid_reduced[gid_reduced['status']=='Closed']

# Remove Crew/Self Generated Graffiti Removal requests
tmp_closed = tmp_closed.drop(tmp_closed[(tmp_closed.case_origin == 'Crew/Self Generated') & 
                             (tmp_closed.service_name == 'Graffiti Removal')].index)

# Calculate timely-closed by service_name
temp_summary = pd.DataFrame()
temp_summary['timely_closure'] = (tmp_closed.groupby('service_name').closed_in_14days.sum() + 
                                  tmp_closed.groupby('service_name').closed_in_45days.sum())
temp_summary['late_closure'] = tmp_closed.groupby('service_name').closed_after_45days.sum()
temp_summary.head()

#temp_summary.to_csv('stats_table_5.csv', header=True)

,timely_closure,late_closure
service_name,,
72 HOUR VIOLATION,22539,344
ACTUATOR EQUIPMENT FAILURE,1,0
ADMIN - REFERRAL REREFERRAL,6,1
AVA,5,0
BICYCLE - TEO - BICYCLE FACILITY LANE,0,1


####  Days_till_closed for a single service_name ("Graffiti Removal" or "Traffic Signal Light Out") vs demographics
The next two cells were for a quick analyses to see if there was any relationship between the amount of time it took to deal with *either* graffiti or a broken traffic signal light, and the demographics of the area originating the request. (Spoiler: no)

In [10]:
# Extract closed requests
tmp_closed = gid_reduced[(gid_reduced['status']=='Closed') &
                         (gid_reduced['service_name']=='Graffiti Removal') &
                         (gid_reduced['closed_after_45days']==0)]

# Remove Crew/Self Generated Graffiti Removal requests
tmp_closed = tmp_closed[tmp_closed['case_origin'] != 'Crew/Self Generated']

# Calculate days_till_closed by zipcode
temp_summary = pd.DataFrame()
temp_summary['days_till_closed'] = tmp_closed.groupby('zipcode').days_till_closed.mean()
temp_summary['load_by_service'] = tmp_closed.groupby('zipcode').load_by_service.mean()
temp_summary['load_by_service_mean'] = tmp_closed.groupby('zipcode').load_by_service_zip.mean()

# Add columns for household income and age, averaged over district
temp_summary['household_income'] = gid_reduced.groupby('zipcode').household_income.mean()
temp_summary['pop_density'] = gid_reduced.groupby('zipcode').pop_density.mean()
temp_summary.head()

#temp_summary.to_csv('stats_table_6.csv', header=True)

,days_till_closed,load_by_service,load_by_service_mean,household_income,pop_density
zipcode,,,,,
91902,NaN,NaN,NaN,76075.0,2024.891030
91911,NaN,NaN,NaN,50186.0,7086.058226
91932,NaN,NaN,NaN,45790.0,10233.983290
91942,NaN,NaN,NaN,49556.0,6525.368529
91945,NaN,NaN,NaN,51932.0,6488.277268


In [11]:
# Extract closed requests
tmp_closed = gid_reduced[(gid_reduced['status']=='Closed') &
                         (gid_reduced['service_name']=='Traffic Signal Light Out') &
                         (gid_reduced['closed_after_45days']==0)]

# Calculate days_till_closed by zipcode
temp_summary = pd.DataFrame()
temp_summary['days_till_closed'] = tmp_closed.groupby('zipcode').days_till_closed.mean()

# Add columns for household income and age, averaged over district
temp_summary['household_income'] = gid_reduced.groupby('zipcode').household_income.mean()
temp_summary['pop_density'] = gid_reduced.groupby('zipcode').pop_density.mean()
temp_summary.head()

#temp_summary.to_csv('stats_table_7.csv', header=True)

,days_till_closed,household_income,pop_density
zipcode,,,
91902,NaN,76075.0,2024.891030
91911,NaN,50186.0,7086.058226
91932,NaN,45790.0,10233.983290
91942,NaN,49556.0,6525.368529
91945,NaN,51932.0,6488.277268
